# LAB 1.3: ADVANCED FEW-SHOT LEARNING

**Course:** Advanced Prompt Engineering Training  
**Session:** Session 1 - Prompt Engineering Fundamentals Review  
**Duration:** 50 minutes  
**Type:** Hands-on Pattern Learning & Example Selection

## LAB OVERVIEW

This lab explores **advanced few-shot learning techniques** for BFSI applications. You'll learn how to:

- Design effective few-shot examples that teach models patterns
- Select optimal examples dynamically based on input
- Use contrastive learning (positive and negative examples)
- Balance example quality vs. quantity
- Apply few-shot learning to complex classification tasks

**Scenario:** You're building an AI system for a bank's customer service department. The system must classify customer intents, assess loan risk from unstructured text, and categorize financial documents. Few-shot learning allows you to teach the model these patterns without fine-tuning.

## LEARNING OBJECTIVES

By the end of this lab, you will be able to:

✓ Design high-quality few-shot examples that teach clear patterns  
✓ Apply contrastive learning (positive + negative examples)  
✓ Implement dynamic example selection based on input similarity  
✓ Optimize example count for accuracy and token efficiency  
✓ Create domain-specific few-shot libraries for BFSI  
✓ Measure and improve few-shot learning effectiveness

### Step 1: Import Libraries

In [1]:
# Lab 1.3: Advanced Few-Shot Learning
# Advanced Prompt Engineering Training - Session 1

import os
import json
from openai import OpenAI
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
from sklearn.metrics.pairwise import cosine_similarity

print("✓ Libraries imported")

✓ Libraries imported


### Step 2: Configure OpenAI Client

In [2]:
# Initialize OpenAI client
api_key=os.environ.get("OPENAI_API_KEY")

# Configuration
MODEL = os.getenv("MODEL_NAME")
TEMPERATURE = 0  # Deterministic for consistent debugging

if not api_key:
    raise ValueError("OPENAI_API_KEY not found. Please set it in .env file")

if not MODEL:
    raise ValueError("MODEL_NAME not found. Please set it in .env file")

client = OpenAI(api_key=api_key)

print(f"✓ Model: {MODEL}")
print(f"✓ Temperature: {TEMPERATURE}")

✓ Model: gpt-4o
✓ Temperature: 0


### Step 3: Create Helper Functions

In [3]:
def call_gpt4(prompt, system_prompt="You are a helpful AI assistant.", temperature=0):
    """
    Wrapper for GPT-4 API calls
    
    Args:
        prompt (str): User prompt
        system_prompt (str): System prompt
        temperature (float): Sampling temperature
    
    Returns:
        dict: Response with content and metadata
    """
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature
        )
        
        return {
            "content": response.choices[0].message.content,
            "total_tokens": response.usage.total_tokens
        }
    except Exception as e:
        return {
            "content": f"Error: {str(e)}",
            "total_tokens": 0
        }

def get_embedding(text: str) -> List[float]:
    """
    Get text embedding for similarity comparison
    
    Args:
        text (str): Text to embed
    
    Returns:
        List[float]: Embedding vector
    """
    try:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Embedding error: {e}")
        return []

def calculate_similarity(text1: str, text2: str) -> float:
    """
    Calculate cosine similarity between two texts
    
    Args:
        text1 (str): First text
        text2 (str): Second text
    
    Returns:
        float: Similarity score (0-1)
    """
    emb1 = get_embedding(text1)
    emb2 = get_embedding(text2)
    
    if not emb1 or not emb2:
        return 0.0
    
    similarity = cosine_similarity([emb1], [emb2])[0][0]
    return float(similarity)

def evaluate_classification(predictions: List[str], ground_truth: List[str]) -> Dict:
    """
    Calculate classification accuracy metrics
    
    Args:
        predictions (List[str]): Predicted labels
        ground_truth (List[str]): True labels
    
    Returns:
        Dict: Accuracy metrics
    """
    correct = sum(1 for pred, true in zip(predictions, ground_truth) if pred.strip().upper() == true.strip().upper())
    total = len(predictions)
    accuracy = correct / total if total > 0 else 0
    
    return {
        "correct": correct,
        "total": total,
        "accuracy": accuracy,
        "accuracy_pct": accuracy * 100
    }

print("✓ Helper functions created")

✓ Helper functions created


In [4]:
# Example: Calculate similarity between two customer queries
query1 = "I want to apply for a personal loan"
query2 = "How can I get a home loan?"
query3 = "What is my account balance?"

# Calculate similarity between similar queries (both about loans)
similarity_score_1 = calculate_similarity(query1, query2)
print(f"Similarity between loan queries: {similarity_score_1:.4f}")

# Calculate similarity between dissimilar queries
similarity_score_2 = calculate_similarity(query1, query3)
print(f"Similarity between loan and balance queries: {similarity_score_2:.4f}")

# Example with financial documents
doc1 = "This customer has a stable income and good credit history"
doc2 = "The applicant shows consistent earnings and excellent credit score"
doc3 = "The transaction was flagged for potential fraud"

similarity_score_3 = calculate_similarity(doc1, doc2)
print(f"Similarity between similar risk assessments: {similarity_score_3:.4f}")

similarity_score_4 = calculate_similarity(doc1, doc3)
print(f"Similarity between risk and fraud texts: {similarity_score_4:.4f}")


Similarity between loan queries: 0.5692
Similarity between loan and balance queries: 0.2500
Similarity between similar risk assessments: 0.6678
Similarity between risk and fraud texts: 0.2934


### Step 4: Test Connection

In [ ]:
# Test connection and embedding function
test = call_gpt4("Say 'Ready for few-shot learning' if you receive this.")
print(f"Response: {test['content']}")

# Test embedding
test_embedding = get_embedding("test sentence")
print(f"Embedding dimension: {len(test_embedding)}")
print("\n✓ Connection verified")

## FEW-SHOT LEARNING THEORY

### What is Few-Shot Learning?

Few-shot learning teaches the model a task by providing examples in the prompt. Instead of explaining rules, you **show** the pattern through examples.

**Zero-shot (No examples):**
```
Classify this email as URGENT or ROUTINE:
"Meeting at 3pm tomorrow"
```

**Few-shot (With examples):**
```
Classify emails as URGENT or ROUTINE:

Example 1: "Server down, customers affected" → URGENT
Example 2: "Meeting at 3pm tomorrow" → ROUTINE
Example 3: "Data breach detected" → URGENT

Now classify: "Weekly report available"
```

### Why Few-Shot Learning Matters for BFSI

1. **Domain-specific patterns** - Banking/finance has unique terminology
2. **No fine-tuning needed** - Fast deployment, no ML expertise required
3. **Adaptable** - Easy to update examples as requirements change
4. **Explainable** - Examples serve as documentation
5. **Token-efficient** - Better than long rule descriptions

## CHALLENGE 1: BASIC FEW-SHOT PATTERNS

**Time:** 10 minutes  
**Objective:** Learn fundamental few-shot pattern construction

### Background

The bank needs to classify customer service inquiries into categories to route them to the right department. Currently using rule-based keywords, but it's failing on edge cases.

### Categories

- **CARD_SERVICES**: Credit/debit card issues, PIN, ATM, card activation
- **LENDING**: Loans, mortgages, credit applications
- **FRAUD**: Suspicious activity, unauthorized transactions
- **ACCOUNT_INQUIRY**: Balance, statements, transaction history
- **GENERAL**: Everything else

### Current Approach (Rule-Based)

In [ ]:
# BASELINE: Rule-based keyword matching (what they're replacing)

def rule_based_classifier(text: str) -> str:
    """Simple keyword-based classification"""
    text_lower = text.lower()
    
    if any(word in text_lower for word in ["card", "atm", "pin", "chip"]):
        return "CARD_SERVICES"
    elif any(word in text_lower for word in ["loan", "mortgage", "credit"]):
        return "LENDING"
    elif any(word in text_lower for word in ["fraud", "suspicious", "unauthorized"]):
        return "FRAUD"
    elif any(word in text_lower for word in ["balance", "statement", "transaction"]):
        return "ACCOUNT_INQUIRY"
    else:
        return "GENERAL"

# Test cases
test_queries = [
    "My card was declined at the store",
    "I want to apply for a home loan",
    "There's a charge I don't recognize",
    "What's my current balance?",
    "Can I get a credit limit increase?"
]

print("RULE-BASED CLASSIFICATION:")
print("=" * 80)
for query in test_queries:
    result = rule_based_classifier(query)
    print(f"Query: {query}")
    print(f"Classification: {result}\n")
print("=" * 80)

**Problems with rule-based:**
- ❌ Last query: "credit limit increase" → classified as LENDING (wrong, should be CARD_SERVICES)
- ❌ Doesn't handle variations or context
- ❌ Requires manual keyword list maintenance
- ❌ Brittle - breaks on unexpected phrasing

### Student Exercise

In [ ]:
# TODO: Create a few-shot classification prompt
# Requirements:
# - Include 2-3 examples per category
# - Clear, diverse examples
# - Consistent format

few_shot_prompt = """
[WRITE YOUR FEW-SHOT PROMPT HERE]

Now classify: {query}
"""

# TODO: Test your prompt
# for query in test_queries:
#     response = call_gpt4(few_shot_prompt.format(query=query))
#     print(f"Query: {query}")
#     print(f"Classification: {response['content']}\n")

### Solution

In [ ]:
# SOLUTION: Well-structured few-shot prompt

few_shot_prompt_v1 = """
Classify customer service inquiries into these categories:
- CARD_SERVICES: Credit/debit card issues
- LENDING: Loans, mortgages, credit applications  
- FRAUD: Suspicious or unauthorized activity
- ACCOUNT_INQUIRY: Balance, statements, transactions
- GENERAL: Other inquiries

Examples:

Input: "My card was declined at the grocery store"
Output: CARD_SERVICES

Input: "I forgot my PIN number"
Output: CARD_SERVICES

Input: "I want to apply for a mortgage"
Output: LENDING

Input: "What interest rate can I get on a car loan?"
Output: LENDING

Input: "There's a $500 charge I didn't make"
Output: FRAUD

Input: "Someone tried to use my card in another country"
Output: FRAUD

Input: "What's my account balance?"
Output: ACCOUNT_INQUIRY

Input: "Can I download my last 3 months of statements?"
Output: ACCOUNT_INQUIRY

Input: "What are your branch hours?"
Output: GENERAL

Now classify this inquiry:
Input: {query}
Output:
"""

system_prompt_v1 = "You are a customer service inquiry classifier. Output only the category name."

# Test the few-shot prompt
print("FEW-SHOT CLASSIFICATION:")
print("=" * 80)

for query in test_queries:
    response = call_gpt4(few_shot_prompt_v1.format(query=query), system_prompt_v1)
    print(f"Query: {query}")
    print(f"Classification: {response['content']}")
    print("-" * 80)

### Validation

In [ ]:
# Compare rule-based vs few-shot

ground_truth = ["CARD_SERVICES", "LENDING", "FRAUD", "ACCOUNT_INQUIRY", "CARD_SERVICES"]

# Rule-based predictions
rule_predictions = [rule_based_classifier(q) for q in test_queries]

# Few-shot predictions
few_shot_predictions = []
for query in test_queries:
    response = call_gpt4(few_shot_prompt_v1.format(query=query), system_prompt_v1)
    few_shot_predictions.append(response['content'].strip())

# Evaluate
rule_metrics = evaluate_classification(rule_predictions, ground_truth)
few_shot_metrics = evaluate_classification(few_shot_predictions, ground_truth)

print("\nCOMPARISON:")
print("=" * 80)
print(f"Rule-based accuracy: {rule_metrics['accuracy_pct']:.1f}% ({rule_metrics['correct']}/{rule_metrics['total']})")
print(f"Few-shot accuracy: {few_shot_metrics['accuracy_pct']:.1f}% ({few_shot_metrics['correct']}/{few_shot_metrics['total']})")
print(f"Improvement: +{few_shot_metrics['accuracy_pct'] - rule_metrics['accuracy_pct']:.1f}%")
print("=" * 80)

### Key Takeaways

✓ **Examples teach patterns** - Model learns from demonstrations  
✓ **Consistent format** - Use same Input/Output structure  
✓ **Coverage** - Include examples for each category  
✓ **Clarity** - Examples should be unambiguous

## CHALLENGE 2: EXAMPLE SELECTION STRATEGY

**Time:** 10 minutes  
**Objective:** Learn which examples matter most

### Background

You have 50+ example customer inquiries. Including all of them would exceed token limits. How do you choose the BEST examples to include?

### Example Library

In [ ]:
# Large library of labeled examples
example_library = [
    # CARD_SERVICES examples
    {"text": "My card was declined", "label": "CARD_SERVICES"},
    {"text": "I lost my credit card", "label": "CARD_SERVICES"},
    {"text": "My chip isn't working", "label": "CARD_SERVICES"},
    {"text": "I need to activate my new card", "label": "CARD_SERVICES"},
    {"text": "ATM didn't give me cash but charged my account", "label": "CARD_SERVICES"},
    {"text": "Can I increase my credit limit?", "label": "CARD_SERVICES"},
    {"text": "My card expired, when will I get a new one?", "label": "CARD_SERVICES"},
    
    # LENDING examples
    {"text": "I want to apply for a mortgage", "label": "LENDING"},
    {"text": "What's the interest rate on personal loans?", "label": "LENDING"},
    {"text": "How much can I borrow for a car?", "label": "LENDING"},
    {"text": "I want to refinance my home", "label": "LENDING"},
    {"text": "What documents do I need for a business loan?", "label": "LENDING"},
    {"text": "Can I get pre-approved for a mortgage?", "label": "LENDING"},
    
    # FRAUD examples
    {"text": "I see charges I didn't make", "label": "FRAUD"},
    {"text": "Someone used my card in another state", "label": "FRAUD"},
    {"text": "There's a suspicious withdrawal", "label": "FRAUD"},
    {"text": "I think my account was hacked", "label": "FRAUD"},
    {"text": "Unauthorized transfer from my account", "label": "FRAUD"},
    
    # ACCOUNT_INQUIRY examples
    {"text": "What's my balance?", "label": "ACCOUNT_INQUIRY"},
    {"text": "Can I get my transaction history?", "label": "ACCOUNT_INQUIRY"},
    {"text": "I need last month's statement", "label": "ACCOUNT_INQUIRY"},
    {"text": "Show me my recent deposits", "label": "ACCOUNT_INQUIRY"},
    {"text": "Why was I charged a fee?", "label": "ACCOUNT_INQUIRY"},
    
    # GENERAL examples
    {"text": "What are your hours?", "label": "GENERAL"},
    {"text": "Where's the nearest branch?", "label": "GENERAL"},
    {"text": "Do you have a mobile app?", "label": "GENERAL"},
    {"text": "How do I update my address?", "label": "GENERAL"}
]

print(f"Example library size: {len(example_library)} examples")
print(f"Token limit challenge: Can only include ~15 examples in prompt")

### Selection Strategies

In [ ]:
# Strategy 1: Random selection
def select_random(examples: List[Dict], n: int = 15) -> List[Dict]:
    """Randomly select n examples"""
    import random
    return random.sample(examples, min(n, len(examples)))

# Strategy 2: Balanced selection (equal per class)
def select_balanced(examples: List[Dict], n_per_class: int = 3) -> List[Dict]:
    """Select n examples from each class"""
    selected = []
    labels = set(ex['label'] for ex in examples)
    
    for label in labels:
        class_examples = [ex for ex in examples if ex['label'] == label]
        selected.extend(class_examples[:n_per_class])
    
    return selected

# Strategy 3: Diverse selection (maximize variety within class)
def select_diverse(examples: List[Dict], n_per_class: int = 3) -> List[Dict]:
    """Select diverse examples from each class using length variation"""
    selected = []
    labels = set(ex['label'] for ex in examples)
    
    for label in labels:
        class_examples = [ex for ex in examples if ex['label'] == label]
        # Sort by length to get variety (short, medium, long)
        class_examples_sorted = sorted(class_examples, key=lambda x: len(x['text']))
        
        # Pick from beginning, middle, end to maximize diversity
        if len(class_examples_sorted) >= n_per_class:
            indices = [0, len(class_examples_sorted)//2, len(class_examples_sorted)-1]
            for i in indices[:n_per_class]:
                selected.append(class_examples_sorted[i])
        else:
            selected.extend(class_examples_sorted)
    
    return selected

print("✓ Selection strategies defined")

### Student Exercise

In [ ]:
# TODO: Test the three strategies and compare accuracy
# Hint: Use the test_queries from Challenge 1 as your test set

# Strategy 1: Random
# Strategy 2: Balanced  
# Strategy 3: Diverse

# Which performs best? Why?

### Solution

In [ ]:
# SOLUTION: Compare selection strategies

def create_few_shot_prompt_from_examples(examples: List[Dict], query: str) -> str:
    """Create few-shot prompt from selected examples"""
    prompt = """Classify customer service inquiries into these categories:
- CARD_SERVICES
- LENDING
- FRAUD
- ACCOUNT_INQUIRY
- GENERAL

Examples:

"""
    for ex in examples:
        prompt += f'Input: "{ex["text"]}"\n'
        prompt += f'Output: {ex["label"]}\n\n'
    
    prompt += f'Now classify:\nInput: "{query}"\nOutput:'
    
    return prompt

# Test data with ground truth
test_data = [
    ("My card was declined at the store", "CARD_SERVICES"),
    ("I want to apply for a home loan", "LENDING"),
    ("There's a charge I don't recognize", "FRAUD"),
    ("What's my current balance?", "ACCOUNT_INQUIRY"),
    ("Can I get a credit limit increase?", "CARD_SERVICES"),
    ("What interest rate for auto loans?", "LENDING"),
    ("Someone withdrew money without permission", "FRAUD"),
    ("I need my tax documents", "ACCOUNT_INQUIRY"),
    ("Where's your nearest ATM?", "GENERAL"),
    ("My PIN is locked", "CARD_SERVICES")
]

system_prompt = "You are a classifier. Output only the category name."

# Test each strategy
strategies = {
    "Random": select_random(example_library, 15),
    "Balanced": select_balanced(example_library, 3),
    "Diverse": select_diverse(example_library, 3)
}

results = {}

print("TESTING SELECTION STRATEGIES:")
print("=" * 80)

for strategy_name, selected_examples in strategies.items():
    print(f"\n{strategy_name} Strategy ({len(selected_examples)} examples):")
    print("-" * 80)
    
    predictions = []
    ground_truth = []
    
    for query, true_label in test_data:
        prompt = create_few_shot_prompt_from_examples(selected_examples, query)
        response = call_gpt4(prompt, system_prompt)
        predicted_label = response['content'].strip()
        
        predictions.append(predicted_label)
        ground_truth.append(true_label)
    
    # Evaluate
    metrics = evaluate_classification(predictions, ground_truth)
    results[strategy_name] = metrics
    
    print(f"Accuracy: {metrics['accuracy_pct']:.1f}% ({metrics['correct']}/{metrics['total']})")
    
    # Show misclassifications
    misclassified = [(test_data[i][0], predictions[i], ground_truth[i]) 
                     for i in range(len(predictions)) 
                     if predictions[i] != ground_truth[i]]
    
    if misclassified:
        print(f"Misclassifications: {len(misclassified)}")
        for query, pred, true in misclassified:
            print(f"  '{query[:40]}...' → Predicted: {pred}, Actual: {true}")
    else:
        print("No misclassifications!")

print("\n" + "=" * 80)
print("\nSTRATEGY COMPARISON:")
print("=" * 80)
for strategy_name, metrics in results.items():
    print(f"{strategy_name:15} - Accuracy: {metrics['accuracy_pct']:.1f}%")
print("=" * 80)

### Key Takeaways

✓ **Balance matters** - Equal examples per class  
✓ **Diversity within classes** - Show variations  
✓ **Quality over quantity** - 3 good examples per class > 10 random  
✓ **Consistency** - Deterministic selection preferable to random

## CHALLENGE 3: CONTRASTIVE EXAMPLES

**Time:** 10 minutes  
**Objective:** Use negative examples to sharpen classification boundaries

### Background

The bank's loan risk classifier struggles with **edge cases** - applications that seem risky but aren't, or seem safe but aren't. Contrastive learning (showing both positive and negative examples) helps the model learn boundaries.

In [ ]:
# These are intentionally ambiguous/tricky
edge_case_applications = [
    {
        "description": "Recent college graduate, employed 6 months, $45K salary, requesting $200K mortgage",
        "true_risk": "HIGH_RISK"  # New to workforce, loan too large
    },
    {
        "description": "Self-employed 10 years, income varies $60K-$120K annually, excellent credit score 780",
        "true_risk": "LOW_RISK"  # Established business, great credit
    },
    {
        "description": "Salaried $90K, 3 years employed, credit score 620, requesting $15K personal loan",
        "true_risk": "HIGH_RISK"  # Low credit score is the issue
    },
    {
        "description": "Retired, fixed income $50K/year, no debt, owns home outright, requesting $10K",
        "true_risk": "LOW_RISK"  # Stable, low debt-to-income
    },
    {
        "description": "Employed 5 years $75K, credit score 720, has 3 existing loans totaling $80K",
        "true_risk": "HIGH_RISK"  # Debt burden too high
    }
]

### Approach 1: Positive Examples Only

In [ ]:
# WITHOUT contrastive learning - only positive examples

positive_only_prompt = """
Classify loan applications as HIGH_RISK or LOW_RISK.

HIGH_RISK examples:
- "Unemployed, poor credit, requesting large loan"
- "Multiple missed payments, high debt-to-income ratio"
- "No credit history, first job, large loan request"

LOW_RISK examples:
- "Stable employment 15 years, excellent credit, low debt"
- "High income, low loan amount, perfect payment history"
- "Owns assets, no debt, conservative loan request"

Classify: {description}
"""

system_prompt_positive = "You are a loan risk assessor. Output only HIGH_RISK or LOW_RISK."

print("POSITIVE-ONLY APPROACH:")
print("=" * 80)

positive_predictions = []
for app in edge_case_applications:
    response = call_gpt4(
        positive_only_prompt.format(description=app['description']),
        system_prompt_positive
    )
    prediction = response['content'].strip()
    positive_predictions.append(prediction)
    
    is_correct = prediction == app['true_risk']
    marker = "✓" if is_correct else "✗"
    
    print(f"\n{marker} {app['description'][:60]}...")
    print(f"  Predicted: {prediction}, Actual: {app['true_risk']}")

# Calculate accuracy
ground_truth = [app['true_risk'] for app in edge_case_applications]
metrics_positive = evaluate_classification(positive_predictions, ground_truth)
print(f"\nAccuracy: {metrics_positive['accuracy_pct']:.1f}%")
print("=" * 80)

### Student Exercise

In [ ]:
# TODO: Create a contrastive learning prompt
# Requirements:
# - Include examples that look similar but have different risk levels
# - Explain why the classification differs
# - Help model learn decision boundaries

contrastive_prompt = """
[WRITE YOUR CONTRASTIVE LEARNING PROMPT HERE]
"""

# TODO: Test and compare to positive-only approach

### Solution

In [ ]:
# SOLUTION: Contrastive learning with boundary examples

contrastive_prompt = """
Classify loan applications as HIGH_RISK or LOW_RISK.

CONTRASTIVE EXAMPLES (learn the boundaries):

Example 1a: "Self-employed 2 years, income varies $30K-$80K, credit score 680"
Risk: HIGH_RISK
Reason: Recent self-employment, high income volatility, marginal credit

Example 1b: "Self-employed 10 years, income varies $60K-$120K, credit score 780"
Risk: LOW_RISK  
Reason: Established business, higher income range, excellent credit

Example 2a: "Recent graduate, 6 months employed, $50K salary, requesting $200K"
Risk: HIGH_RISK
Reason: Short employment history, loan far exceeds annual income

Example 2b: "Recent graduate, 2 years employed, $55K salary, requesting $25K"
Risk: LOW_RISK
Reason: Reasonable employment duration, loan is manageable relative to income

Example 3a: "Employed 5 years, $80K salary, credit score 625, requesting $20K"
Risk: HIGH_RISK
Reason: Credit score below acceptable threshold (650+) despite good income

Example 3b: "Employed 5 years, $80K salary, credit score 720, requesting $20K"
Risk: LOW_RISK
Reason: Good credit, stable employment, appropriate loan size

Example 4a: "Retired, $40K fixed income, no savings, requesting $15K"
Risk: HIGH_RISK
Reason: Fixed income with no buffer, loan is significant portion of annual income

Example 4b: "Retired, $50K fixed income, no debt, owns home, requesting $10K"
Risk: LOW_RISK
Reason: Stable income, assets, low debt burden, conservative loan

KEY DECISION FACTORS:
- Credit score: <650 = high risk concern
- Employment: <2 years = risk factor
- Debt-to-income: Existing debt >40% of income = high risk
- Loan size: >4x annual income = high risk

Now classify: {description}
Output format: [HIGH_RISK or LOW_RISK]
"""

system_prompt_contrastive = """You are a loan risk assessor trained on contrastive examples.
You carefully consider boundary cases.
You output only HIGH_RISK or LOW_RISK."""

print("CONTRASTIVE LEARNING APPROACH:")
print("=" * 80)

contrastive_predictions = []
for app in edge_case_applications:
    response = call_gpt4(
        contrastive_prompt.format(description=app['description']),
        system_prompt_contrastive
    )
    prediction = response['content'].strip().replace('[','').replace(']','')
    contrastive_predictions.append(prediction)
    
    is_correct = prediction == app['true_risk']
    marker = "✓" if is_correct else "✗"
    
    print(f"\n{marker} {app['description'][:60]}...")
    print(f"  Predicted: {prediction}, Actual: {app['true_risk']}")

# Calculate accuracy
metrics_contrastive = evaluate_classification(contrastive_predictions, ground_truth)
print(f"\nAccuracy: {metrics_contrastive['accuracy_pct']:.1f}%")
print("=" * 80)

### Comparison

In [ ]:
# Compare both approaches

print("\nCOMPARISON:")
print("=" * 80)
print(f"Positive-only accuracy:  {metrics_positive['accuracy_pct']:.1f}% ({metrics_positive['correct']}/{metrics_positive['total']})")
print(f"Contrastive accuracy:    {metrics_contrastive['accuracy_pct']:.1f}% ({metrics_contrastive['correct']}/{metrics_contrastive['total']})")
print(f"Improvement:             +{metrics_contrastive['accuracy_pct'] - metrics_positive['accuracy_pct']:.1f}%")
print("\n" + "=" * 80)

print("\nWHY CONTRASTIVE LEARNING WORKS:")
print("=" * 80)
print("✓ Shows what makes similar cases different")
print("✓ Teaches decision boundaries explicitly")
print("✓ Provides reasoning for classifications")
print("✓ Helps model discriminate edge cases")
print("✓ Reduces false positives and false negatives")
print("=" * 80)

### Key Takeaways

✓ **Contrastive pairs** - Show similar inputs with different outputs  
✓ **Boundary learning** - Teach what separates classes  
✓ **Reasoning** - Include WHY examples differ  
✓ **Edge cases** - Perfect for ambiguous situations

💡 **When to Use Contrastive Learning:**
- Classifications with fuzzy boundaries
- High stakes where false positives/negatives are costly
- Complex decision criteria
- Models struggling with edge cases

## LAB SUMMARY

### Few-Shot Learning Techniques Mastered

| Challenge | Technique | Key Benefit | Best For |
|-----------|-----------|-------------|----------|
| 1 | Basic few-shot | Simple pattern teaching | Clear, distinct categories |
| 2 | Balanced selection | Equal class representation | Multi-class problems |
| 3 | Contrastive learning | Sharp decision boundaries | Edge cases, ambiguous inputs |

### Production Checklist

Before deploying few-shot learning:

- [ ] Examples are high-quality (unambiguous, diverse)
- [ ] All classes have equal representation
- [ ] Contrastive pairs for boundary cases (if needed)
- [ ] Tested multiple example counts
- [ ] Measured accuracy vs. token cost trade-off
- [ ] Examples documented and version-controlled
- [ ] Edge cases covered

